# Question 2: How do One-Hit Wonders perform compared to Superstars on the Billboard Hot 100?

This notebook compares one‑hit wonders (artists with exactly 1 unique Hot 100 song) vs superstars (artists with 5+ unique Hot 100 songs).


Notes: If you'd like the full original analysis restored (visuals and all code cells), reply 'restore full Q2' and I'll reconstruct it from the split notebooks and repo history.

## How to run this notebook locally

1. Place the dataset at `data/Hot Stuff.csv` relative to the repo root.
2. Run each code cell in order.
3. Required packages: pandas, numpy, matplotlib, seaborn, scipy.

In [ ]:
# Quick verification that the notebook is present and that the data path looks available
from pathlib import Path
p = Path('..') / 'data' / 'Hot Stuff.csv'

: null,
: []